# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


### Récupération des données

In [2]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [69]:

function SolveKnapInstance(filename)

    # Récupération des données
    price, weight, capacity = readKnaptxtInstance(filename)

    BestProfit = -1
    
    # Matrice des solutions / Matrice composition
    tab = zeros(length(weight)+1, capacity+1)
    
    for i in 1:length(weight)
        for j in 1:capacity
            if j >= weight[i]
                tab[i+1,j+1] = max(tab[i, j+1],tab[i, j+1-weight[i]]+price[i])
            else
                tab[i+1,j+1] = tab[i,j+1]
            end
        end
    end

    BestProfit = tab[length(weight)+1,capacity+1]

    # Récupération de Bestsol
    Bestsol = zeros(length(weight))
    Val = BestProfit
    i = length(weight)+1
    j = capacity+1

    while Val > 0 && j > 0
        if Val == tab[i-1,j]
            Bestsol[i-1] = 0
            i = i - 1
            j = j
            Val = tab[i,j]
            
        else
            Bestsol[i-1] = 1
            j = j - weight[i-1]
            i = i - 1
            Val = tab[i,j]

        end

    end

    return tab, BestProfit, Bestsol

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [70]:
tab, BestProfit, Bestsol = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
show(stdout, "text/plain", tab)


******

Optimal value = 65.0

Optimal x=[0.0, 1.0, 0.0, 1.0]
5×11 Matrix{Float64}:
 0.0  0.0  0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0
 0.0  0.0  0.0   0.0   0.0   0.0   0.0  42.0  42.0  42.0  42.0
 0.0  0.0  0.0   0.0  40.0  40.0  40.0  42.0  42.0  42.0  42.0
 0.0  0.0  0.0  12.0  40.0  40.0  40.0  52.0  52.0  52.0  54.0
 0.0  0.0  0.0  12.0  40.0  40.0  40.0  52.0  52.0  65.0  65.0

## Questions préliminaires:
1. Quelle est la règle de séparation ? \
   La règle de séparation est celle par ordre lexicographique. On choisit le premier objet dans la liste qui n'a pas été encore pris.

2. Quelle est la méthode de calcul de borne supérieure ? \
   La méthode de calcul de borne supérieure correspond à la solution de la relaxation linéaire en fixant la valeur des objets pris à 1(et celle de objets non pris à 0).

3. Quels sont les tests de sondabilité TA, TO, TR?
   Test de sondabilité: 
   * TA : Le test d'admissibilité passe si la relaxation linéaire admet une solution.
   * TO : La solution de la relaxation linéaire est inférieur au profit maximal.
   * TR : La solution de relaxation linéaire est atteinte par des valeurs entières.

   
4. Quelle est la stratégie d’exploration ? \
   La stratégie d'exploration est la stratégie depthfirst, exploration en profondeur.

## Code et analyse

1. cf code

2. cf code

3. Points clés de  notre implémentation des différents blocs du Branch-and-Bound:

    * `règle de séparation` : nous avons gardé l'ordre lexicographique défini initialement.
    * `TA` : Le test de faisabilité passe si la capacité du sac à dos est négative: en effet le sac à dos déborde et la situation n'est plus valide.
    * `TO` : Le test d'optimalité passe si la valeur de la borne supérieur, obtenue en remplissant le sac à dos, est inférieure à la meilleure solution déjà trouvée pour le sac à dos. En effet dans ce cas, il n'existe aucune combinaison d'objets permettant d'obtenir une meilleure valeur pour le sac final.
    * `TR` : Le test de relaxation passe si nous trouvons une solution évidente au problème. Nous avons choisi de le valider si la capacité du sac à dos est nulle au noeud sondé (ie nous ne pouvons ajouter aucun autre objet dans cette situation, donc nous avons une solution évidente).
    * `Stratégie d'exploration` : nous avons gardé la stratégie deephtfirst (d'exploration en profondeur) définie initialement.
    

4. Pour la structure de données permettant de garder les informations nécessaires au Branch-and-Bound, nous avons choisi de garder la structure définie intialement à l'aide de `listvals`, `listobj`et `listnodes`, ainsi que `BestProfit` et `Bestsol`. Nous avons décidé de mettre à jour ces derniers dans le programme principal à chaque noeud où la capacité est valide. Enfin, nous avons crée `weight_courant`, `price_courant` et `capacity_courant` pour avoir à chaque itération des valeurs actualisées de respectivement `weight`, `price` et `capacity`, qui sont utilisés notamment pour le calcul de la bornesup et déterminer si une nouvelle solution optimale est atteinte.

5. Le résultat du Branch-and-Borne en utilisant quelle que soit la borne, ce qui semble cohérent (d'après le TD). Cependant, selon les bornes, les performances diffèrent. En effet, on remarque que la borne 2 a tendance à être plus petite que la borne 1. Par conséquent, le test d'optimalité a plus tendance à être vrai pour la borne 2 que la borne 1. Ainsi, il y a moins de sépération de noeuds et donc un nombre total de noeuds inférieur en utilisant la borne 2 que la borne 1. 
